In [1]:
from langchain_community.chat_models import ChatOllama  # 使用 Ollama 封裝的 LLaMA 模型
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate
import torch


# 使用本地 LLaMA 模型
llm_llama3_8B = ChatOllama(model="llama3:8B")
llm_phi3_3dot8B = ChatOllama(model="phi3:3.8B")
llm_mistral_7B = ChatOllama(model="mistral:7B")
llm_gemma3_4B = ChatOllama(model="gemma3:4B")

# GPU 加速
print("=== PyTorch GPU 加速環境檢查 ===")
print(f"PyTorch 版本: {torch.__version__}")
print(f"PyTorch 編譯的 CUDA 版本: {torch.version.cuda}")
print(f"是否支援 CUDA: {torch.cuda.is_available()}")

try:
    if torch.cuda.is_available():
        gpu_count = torch.cuda.device_count()
        current_gpu = torch.cuda.current_device()
        device_name = torch.cuda.get_device_name(current_gpu)

        print(f"偵測到 {gpu_count} 個 GPU")
        print(f"當前使用的 GPU：{device_name}")
        device = torch.device("cuda")
    else:
        raise RuntimeError("CUDA 不可用，將使用 CPU")
except Exception as e:
    print(f"無法使用 GPU：{e}")
    device = torch.device("cpu")

print(f"pytorch可用的GPU為：{device}")


c:\Users\TUF\Documents\base-agnet\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


=== PyTorch GPU 加速環境檢查 ===
PyTorch 版本: 2.8.0+cpu
PyTorch 編譯的 CUDA 版本: None
是否支援 CUDA: False
無法使用 GPU：CUDA 不可用，將使用 CPU
pytorch可用的GPU為：cpu


C:\Users\TUF\AppData\Local\Temp\ipykernel_13060\3167653020.py:10: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm_llama3_8B = ChatOllama(model="llama3:8B")


In [2]:
%load_ext autoreload
%autoreload 2

### 建立工具 ###

In [3]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from memory import Memory, MemoryStream
from reflection import ReflectionEngine

# 所有 Agent 共用同一個 MemoryStream，彼此回覆會被寫回 FAISS，後續查詢時能互相參考。
# === 1. 建立共享記憶流 ===
memory_stream = MemoryStream()

# === 2. 封裝記憶檢索工具 ===
memory_tool = Tool(
    name="MemoryRetriever",
    func=lambda q: "\n".join(m.content for m in memory_stream.search_by_vector(q)),
    description="檢索過往的重要對話與事件"
)

# initialize_agent 的 tools 參數必須是 List[Tool]
tools = [memory_tool]


c:\Users\TUF\Documents\base-agnet\memory.py:24: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
c:\Users\TUF\Documents\base-agnet\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 建立 Agent ###

In [ ]:
# === 3. 定義多個角色與對應 LLM ===
# 新增角色，加入新的 (角色描述, llm) 配對
roles = {
    "llama":  ("農夫", llm_llama3_8B),
    "mistral":("植物疾病學者", llm_mistral_7B),
    "gemma":  ("氣候專家", llm_gemma3_4B)
}

agents = {}
for name, (role_desc, llm_model) in roles.items():
    prompt_prefix = f"你是一位{role_desc}，回答時請保持此角色的語氣與思考方式。"
    
    agents[name] = initialize_agent(
        tools=tools,
        llm=llm_model,
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        verbose=True,
        agent_kwargs={
            "prefix": prompt_prefix,
            }
    )


C:\Users\TUF\AppData\Local\Temp\ipykernel_13060\2992608666.py:13: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agents[name] = initialize_agent(


In [5]:
# === 4. 對話流程示例 ===
def multi_agent_round(user_input: str):
    print(f"\n=== 使用者輸入 ===\n{user_input}\n")
    # 對每個 agent 輪詢
    for name, agent in agents.items():
        response = agent.run({
            "input": user_input,
            "chat_history": []
        })
        print(f"[{name}] {response}\n")
        # 將回覆寫入共享記憶
        memory_stream.add_memory(
            f"{name} 回覆: {response}",
            importance=5
        )


In [7]:
# === 5. 測試對話 ===
# 如果是 .py 就要加判斷式
# if __name__ == "__main__":
#     multi_agent_round("請大家一起討論氣候變遷對農業的影響。")
#     multi_agent_round("根據剛剛的討論，再給我一段結論。")
multi_agent_round("請大家一起討論氣候變遷對農業的影響。")
multi_agent_round("根據剛剛的討論，再給我一段結論。")


=== 使用者輸入 ===
請大家一起討論氣候變遷對農業的影響。



> Entering new AgentExecutor chain...
```json
{
  "action": "MemoryRetriever",
  "action_input": "previous discussions about climate change"
}
```

Please use the MemoryRetriever tool to retrieve relevant information from past conversations regarding climate change's impact on agriculture. This will help me better understand the context and provide a more informed response.
Observation: llama 回覆: Climate change has significant impacts on agriculture, including changes in temperature and precipitation patterns, increased frequency and severity of extreme weather events, and altered growing seasons. This can lead to crop failures, reduced yields, and disruptions to food supply chains.
gemma 回覆: Climate change is already impacting agriculture significantly. Rising temperatures are shortening growing seasons and reducing yields, while altered precipitation patterns are impacting soil moisture and increasing the risk of drought or flooding. These changes

In [8]:
# === 6. 反思與長期記憶整理 ===
reflector = ReflectionEngine(llm=llm_llama3_8B,
                             memory_stream=memory_stream,
                             reflection_threshold=20)
reflector.reflect()

已完成反思，新增記憶：
 - 以下是系統近期的重要記憶，3 個高層次洞察：
 - 氣候變化對農業的影響無法低估，溫度和降水模式的改變、極端天氣事件的增加頻率和嚴重性等因素將導致作物失敗、產量下降和糧食供應鏈中的干擾。
 - 氣候變化對農業的影響不僅限於溫度和降水模式的改變，還包括極端天氣事件的增加頻率和嚴重性、作物成熟速度的增加、土壤濕度水平的變化等因素，這些變化將對全球糧食安全產生嚴重的威脅。
 - 为了應對氣候變化的影響，農業需要進行適配，包括開發耐干旱作物、實施灌溜系統管理水資源、開發早期警示系統等策略，以減少氣候變化對農業的影響。但是，這些策略仍然不能完全避免氣候變化對農業的影響。


['以下是系統近期的重要記憶，3 個高層次洞察：',
 '氣候變化對農業的影響無法低估，溫度和降水模式的改變、極端天氣事件的增加頻率和嚴重性等因素將導致作物失敗、產量下降和糧食供應鏈中的干擾。',
 '氣候變化對農業的影響不僅限於溫度和降水模式的改變，還包括極端天氣事件的增加頻率和嚴重性、作物成熟速度的增加、土壤濕度水平的變化等因素，這些變化將對全球糧食安全產生嚴重的威脅。',
 '为了應對氣候變化的影響，農業需要進行適配，包括開發耐干旱作物、實施灌溜系統管理水資源、開發早期警示系統等策略，以減少氣候變化對農業的影響。但是，這些策略仍然不能完全避免氣候變化對農業的影響。']

In [ ]:
from conversation_loop import ConversationLoop

# 建立對話迴圈物件
conv_loop = ConversationLoop(
    agents=agents,
    memory_stream=memory_stream,
    reflector=reflector,
    reflection_interval=2   # 每兩輪觸發一次反思
)

# 啟動對話流程
conv_loop.run_round("請大家一起討論氣候變遷對農業的影響。")
conv_loop.run_round("根據剛剛的討論，再給我一段結論。")
conv_loop.run_round("請提出可行的調適策略。")  # 這一輪會觸發反思